In [ ]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report

In [ ]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

Choose a project among QEMU and FFMPEG

In [ ]:
#data = pd.read_csv(os.path.join('..','data', 'ffmpeg_data_reduced.csv'))
data = pd.read_csv(os.path.join('..','data', 'qemu_data_reduced.csv'))

In [ ]:
print(data["target"].value_counts())
max_length = data['length'].max()
print(max_length)

PRE_PROCESSING

Word2Vec NL pre-trained embeddings

In [158]:
# from gensim.test.utils import common_texts
# from gensim.models import Word2Vec

# model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
# model.save("word2vec.model")

In [159]:
import gensim.downloader
# w2v_vectors = gensim.downloader.load('word2vec-google-news-300')

In [160]:
#w2v_vectors.most_similar('facebook')

In [161]:
import nltk
from nltk.tokenize import word_tokenize

# Download the Punkt tokenizer models if not already downloaded
nltk.download('punkt')

tokenized_list = [word_tokenize(sentence) for sentence in data["func"].tolist()]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iliaskaloup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [162]:
# Encode each token using Word2Vec embeddings
encoded_list = []
for sentence_tokens in tokenized_list:
    encoded_sentence = []
    for token in sentence_tokens:
        if token in w2v_vectors:
            encoded_token = w2v_vectors[token]
            encoded_sentence.append(encoded_token)
    encoded_list.append(encoded_sentence)

data["w2v_func"] = encoded_list

FastText NL pre-trained embeddings

In [163]:
from gensim.models import FastText
#fastText_vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')

In [164]:
f_tokenized_list = [word_tokenize(sentence) for sentence in data["func"].tolist()]

# Encode each token using fastText embeddings
encoded_list = []
for sentence_tokens in f_tokenized_list:
    encoded_sentence = []
    for token in sentence_tokens:
        if token in fastText_vectors:
            encoded_token = fastText_vectors[token]
            encoded_sentence.append(encoded_token)
        else:
            continue
    encoded_list.append(encoded_sentence)

data["ft_func"] = encoded_list

BERT pre-trained embeddings

In [165]:
model_variation = "bert-base-uncased" # "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_variation)
bert = TFAutoModel.from_pretrained(model_variation)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [166]:
bert_embeddings = bert.get_input_embeddings()
embedding_matrix = bert_embeddings.weights[0].numpy()
num_words = len(embedding_matrix)
print(num_words)
dim = len(embedding_matrix[0])
print(dim)

30522
768


In [167]:
sequences = [tokenizer.encode(sente, padding=True, truncation=True, add_special_tokens=False, return_tensors="tf").numpy() for sente in data["func"]] # Tokenize the complete sentences

lines_pad = []
for seq in sequences:
    lines_pad.append(seq[0])


In [168]:
data["bert_func"] = lines_pad

CodeBERT pre-trained CPP embeddings

In [169]:
model_variation = "microsoft/codebert-base" # "neulab/codebert-cpp"
tokenizer = AutoTokenizer.from_pretrained(model_variation, do_lower_case=False) # , do_lower_case=True
codebert = TFAutoModel.from_pretrained(model_variation) # , from_pt=True

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [170]:
codebert_embeddings = codebert.get_input_embeddings()
embedding_matrix = codebert_embeddings.weights[0].numpy()
num_words = len(embedding_matrix)
print(num_words)
dim = len(embedding_matrix[0])
print(dim)

50265
768


In [171]:
sequences = [tokenizer(sente, return_tensors="tf", padding=True, truncation=True, add_special_tokens=False) for sente in data["func"]]

In [172]:
def extractEmbList(sequences):
    lines_pad = []
    for sequence in sequences:
        seq = sequence['input_ids'].numpy()[0]
#         if len(seq) < max_len:
#             for i in range(len(seq), max_len):
#                 seq = np.append(seq, 0)
        lines_pad.append(seq)
    
    [arr.tolist() for arr in lines_pad]
    
    return lines_pad

lines_pad = np.array(extractEmbList(sequences))

C:\Users\iliaskaloup\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [173]:
data["codebert_func"] = lines_pad

Training

Evaluation